**Predicting Smoke during Surgeries using non CNN method**

**Hue Saturation Value Approach**

In [5]:
# import statements
import os
import cv2
import numpy as np
import time

In [6]:
# preprocessing for input images
def detect_black_border(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)[1]
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]
    x, y, w, h = cv2.boundingRect(largest_contour)
    shrink_factor = 0.05
    x += int(w * shrink_factor)
    y += int(h * shrink_factor)
    w = int(w * (1 - 2 * shrink_factor))
    h = int(h * (1 - 2 * shrink_factor))
    return x, y, w, h

def crop_and_resize(image, x, y, w, h, target_size=(224, 224)):
    bbox = (x, y, x + w, y + h)
    cropped_image = image[y:y+h, x:x+w]
    resized_image = cv2.resize(cropped_image, target_size, interpolation=cv2.INTER_LINEAR)
    return resized_image


# detect smoke function to detect smoke using HSV thresholding techniques
def detect_smoke(image):
    blur = cv2.GaussianBlur(image, (21, 21), 0)
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    lower = np.array([0, 0, 50], dtype="uint8")
    upper = np.array([179, 50, 255], dtype="uint8")
    mask = cv2.inRange(hsv, lower, upper)
    no_smoke_pixels = cv2.countNonZero(mask)
    
    # Threshold of 12600 pixels has been set
    return no_smoke_pixels > 12600

data_dir = '/kaggle/input/ps2test-lc/PS2 Test'
output_file = 'predictions-hsv.csv'

# start time to calculate inference time for hsv
start_time = time.time() 

with open(output_file, 'w') as csv_file:
    csv_file.write('Video,Sample,Smoke\n')
    
    for video in os.listdir(data_dir):
        video_dir = os.path.join(data_dir, video)
        
        for sample in os.listdir(video_dir):
            sample_dir = os.path.join(video_dir, sample)
            sample_smoke_detected = False
            
            for frame_name in os.listdir(sample_dir):
                frame_path = os.path.join(sample_dir, frame_name)
                frame = cv2.imread(frame_path)
                
                if frame is not None:
                    smoke_detected = detect_smoke(frame)

                    # If smoke is detected in any frame, mark the sample as having smoke
                    if smoke_detected:
                        sample_smoke_detected = True

            # Write result to CSV file for the sample
            csv_file.write(f'{video},{sample},{int(sample_smoke_detected)}\n')

end_time = time.time() 

runtime_seconds = end_time - start_time
print("Total runtime (seconds):", runtime_seconds)
print("Results saved to:", output_file)


Total runtime (seconds): 99.4908127784729
Results saved to: predictions-hsv.csv
